# Historical Data

In [ ]:
import pandas as pd

In [417]:
historical_data = pd.read_csv('/Users/monmac/Desktop/448f97988afe977af36cfc73cde95211.csv')

In [459]:
historical_data.isnull().sum();

In [437]:
pd.to_datetime(1629356400, unit='s')

Timestamp('2021-08-19 07:00:00')

In [421]:
historical_data['rain_1h'].isnull().sum()

90637

In [458]:
historical_data['weather_main'].value_counts();

In [423]:
possible_conditions = ['Clear', 'Clouds', 'Rain', 'Mist', 'Drizzle', 'Fog', 'Snow', 'Thunderstorm', 'Haze']

In [457]:
historical_data['weather_description'];

In [456]:
historical_data.dtypes;

In [441]:
historical_data['dt'] = pd.to_datetime(historical_data['dt'], unit='s')
historical_data = historical_data.set_index('dt')

In [452]:
lunch = historical_data[historical_data.index[:].hour==12]
dinner = historical_data[historical_data.index[:].hour==18]
historical_services = pd.concat([lunch, dinner])
historical_services = historical_services.sort_values('dt')

In [460]:
historical_services;

#   Forecast using API

In [484]:
import requests

def fetch_data(api,city,nodays):
    """
    Get weather data from weather API. Returns empty string if data not found'

    """
    #api = '7aa7fc7be1af4500882144440212208'
    #city = 'Paris'
    #nodays = '7'
    url = f'http://api.weatherapi.com/v1/forecast.json?key={api}&q={city}&days={nodays}&aqi=no&alerts=no'

    response = requests.get(url)
    
    if response.status_code != 200:
        return ''
    data = response.json()
    return data['forecast']['forecastday']

In [485]:
data=fetch_data('7aa7fc7be1af4500882144440212208','Paris','3')

In [486]:
daily_data = pd.DataFrame(data[0]['day']).iloc[2,:]
pd.DataFrame(daily_data).head(3)

,text
maxtemp_c,22.2
maxtemp_f,72.0
mintemp_c,15.7


In [472]:
### gives the weather hour by hour for the entire first day###

daily_hourly_data = pd.DataFrame(data[0]['hour'])
#condition has a code and an image, the code below only keeps the needed weather state
for i in range(len(daily_hourly_data)):
    daily_hourly_data['condition'][i]=daily_hourly_data['condition'][i]['text']

/var/folders/p2/6ddwj2812811pbqjm8t6brv80000gn/T/ipykernel_67780/332607310.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_hourly_data['condition'][i]=daily_hourly_data['condition'][i]['text']


In [473]:
#condition is given as three values this keeps only one
data_series=[]
for i in range(3):
    data_series.append(data[i]['hour'])

#condition has a code and an image, the code below only keeps the needed weather state
for x in range(3):
    for y in range(24):
        data_series[x][y]['condition']=data_series[x][y]['condition']['text']

d={}
for i in range(len(data_series)):
    d[i] = pd.DataFrame(data_series[i])

seven_day_forecast = pd.concat([d[i] for i in range(len(data_series))])

In [474]:
seven_day_forecast=seven_day_forecast.reset_index().drop(columns='index')

In [475]:
#convert time column to datetime
seven_day_forecast['time']=pd.to_datetime(seven_day_forecast['time'])

In [476]:
seven_day_forecast['condition'].value_counts()

Partly cloudy        39
Clear                10
Overcast              9
Sunny                 8
Light rain shower     3
Cloudy                3
Name: condition, dtype: int64

In [477]:
pd.read_csv('/Users/monmac/Desktop/weather_conditions.csv');

In [478]:
Clear = ['Clear', 'Sunny']
Clouds = ['Clouds','Partly cloudy', 'Cloudy', 'Overcast']
Rain = ['Rain','Patchy rain possible', 'Patchy light rain', 'Light rain', 'Moderate rain at times','Moderate rain','Heavy rain at times','Heavy rain','Moderate or heavy rain shower','Torrential rain shower']
Mist = ['Mist']
Drizzle = ['Drizzle','Patchy light drizzle','Light drizzle','Freezing drizzle','Heavy freezing drizzle','Light rain shower']
Fog = ['Fog', 'Freezing Fog']
Snow = ['Snow','Patchy snow possible', 'Patchy sleet possible', 'Patchy freezing drizzle possible', 'Blowing snow','Blizzard','Moderate or heavy freezing rain','Light freezing rain','Light sleet', 'Moderate or heavy sleet','Patchy light snow','Light snow','Patchy moderate snow','Moderate snow','Patchy heavy snow','Heavy snow','Ice pellets','Light sleet showers', 'Moderate or heavy sleet showers','Light snow showers','Moderate or heavy snow showers','Light showers of ice pellets','Moderate or heavy showers of ice pellets']
Thunderstorm = ['Thunderstorm','Thundery outbreaks possible', 'Patchy light rain with thunder','Moderate or heavy rain with thunder','Patchy light snow with thunder','Moderate or heavy snow with thunder']
Haze = ['Haze']

In [479]:
summed_conditions = [Clear, Clouds, Rain, Mist, Drizzle, Fog, Snow, Thunderstorm, Haze]

In [480]:
#boolean indexing to decrease possibilities of weather condition
for i in range(len(seven_day_forecast['condition'])):
    for x in summed_conditions: 
        if seven_day_forecast['condition'][i] in x: 
            seven_day_forecast['condition'][i]=x[0]

/var/folders/p2/6ddwj2812811pbqjm8t6brv80000gn/T/ipykernel_67780/3313053615.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seven_day_forecast['condition'][i]=x[0]


In [481]:
seven_day_forecast=seven_day_forecast.set_index('time')

In [482]:
lunch = seven_day_forecast[seven_day_forecast.index[:].hour==12]
dinner = seven_day_forecast[seven_day_forecast.index[:].hour==18]
forecasted_services = pd.concat([lunch, dinner])
forecasted_services = forecasted_services.sort_values('time')

In [483]:
forecasted_services

,time_epoch,temp_c,temp_f,is_day,condition,wind_mph,wind_kph,wind_degree,wind_dir,pressure_mb,...,dewpoint_f,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv
time,,,,,,,,,,,,,,,,,,,,,
2021-08-23 12:00:00,1629712800,18.4,65.1,1,Clouds,8.1,13.0,356,N,1024.0,...,57.4,0,0,0,0,10.0,6.0,9.8,15.8,4.0
2021-08-23 18:00:00,1629734400,21.6,70.9,1,Drizzle,10.1,16.2,34,NE,1023.0,...,56.1,0,32,0,0,10.0,6.0,13.2,21.2,5.0
2021-08-24 12:00:00,1629799200,23.0,73.4,1,Clouds,12.1,19.4,53,NE,1023.0,...,48.6,0,0,0,0,10.0,6.0,13.6,22.0,5.0
2021-08-24 18:00:00,1629820800,23.0,73.4,1,Clouds,12.5,20.2,49,NE,1022.0,...,44.1,0,0,0,0,10.0,6.0,14.8,23.8,6.0
2021-08-25 12:00:00,1629885600,22.8,73.0,1,Clouds,11.9,19.1,53,NE,1021.0,...,45.7,0,0,0,0,10.0,6.0,13.6,22.0,6.0
2021-08-25 18:00:00,1629907200,23.4,74.1,1,Clouds,11.0,17.6,40,NE,1020.0,...,48.0,0,0,0,0,10.0,6.0,13.2,21.2,6.0


# Historical Forecast